In [1]:
import os


In [2]:
!pwd

/Users/krushna/Kidney-Disease-Classification/research


In [3]:
os.chdir("../")

In [4]:
!pwd

/Users/krushna/Kidney-Disease-Classification


In [5]:
#!pip install dagshub
!pip install python-dotenv


In [6]:
import os
from dotenv import load_dotenv
import dagshub
import mlflow

load_dotenv()

repo_owner = os.getenv('DAGSHUB_REPO_OWNER')
repo_name = os.getenv('DAGSHUB_REPO_NAME')

# Check if variables are loaded properly
print(f"Repo Owner: {repo_owner}")
print(f"Repo Name: {repo_name}")

# Initialize DagsHub repo and MLflow
dagshub.init(
    repo_owner=repo_owner, 
    repo_name=repo_name, 
    mlflow=True, 
    url=f"https://dagshub.com/{repo_owner}/{repo_name}.mlflow"
)

# Start an MLflow run and log parameters/metrics
with mlflow.start_run():
    mlflow.log_param('parameter name', 'value')
    mlflow.log_metric('metric name', 1)


Repo Owner: kru2710shna
Repo Name: Kidney-Disease-Classification


Accessing as kru2710shna

Initialized MLflow to track repo "kru2710shna/Kidney-Disease-Classification.mlflow"

Repository kru2710shna/Kidney-Disease-Classification.mlflow initialized!

2024/10/06 03:20:00 INFO mlflow.tracking._tracking_service.client: 🏃 View run intelligent-bass-932 at: https://dagshub.com/kru2710shna/Kidney-Disease-Classification.mlflow.mlflow/#/experiments/0/runs/a4e83ad3e31b4e228a28c25196007644.
2024/10/06 03:20:00 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/kru2710shna/Kidney-Disease-Classification.mlflow.mlflow/#/experiments/0.


In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from Kidney_Image_Classifier.constants import *
from Kidney_Image_Classifier.utils.overlapping import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Kidney_Image_Classification_Data",
            mlflow_uri="https://dagshub.com/kru2710shna/Kidney-Disease-Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
    )
        return eval_config

In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)  # Fix here
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)  # Correctly set the tracking URI
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    
        with mlflow.start_run():
            print("Starting MLflow run...")
            
            # Log parameters
            mlflow.log_params(self.config.all_params)
            print("Logged parameters.")
            
            # Log metrics
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
            print(f"Logged metrics: Loss - {self.score[0]}, Accuracy - {self.score[1]}")
    
            # Model registration
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                print("Logged model with registration.")
            else:
                mlflow.keras.log_model(self.model, "model")
                print("Logged model without registration.")

In [12]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-10-06 03:20:03,985: INFO: overlapping: yaml file: config/config.yaml loaded successfully]
[2024-10-06 03:20:03,986: INFO: overlapping: yaml file: params.yaml loaded successfully]
[2024-10-06 03:20:03,987: INFO: overlapping: created directory at: artifacts]
[2024-10-06 03:20:04,142: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 2207 images belonging to 2 classes.


/Users/krushna/.pyenv/versions/3.10.0/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


148/148 ━━━━━━━━━━━━━━━━━━━━ 220s 1s/step - accuracy: 0.9366 - loss: 0.1570
[2024-10-06 03:23:44,779: INFO: overlapping: json file saved at: scores.json]
Starting MLflow run...


2024/10/06 03:23:45 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


Logged parameters.
Logged metrics: Loss - 0.4083787500858307, Accuracy - 0.817852258682251


/Users/krushna/.pyenv/versions/3.10.0/lib/python3.10/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/10/06 03:24:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 5
Created version '5' of model 'VGG16Model'.
2024/10/06 03:24:16 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/10/06 03:24:16 INFO mlflow.tracking._tracking_service.client: 🏃 View run capable-moose-768 at: https://dagshub.com/kru2710shna/Kidney-Disease-Classificatio

Logged model with registration.


In [ ]:
!mlflow ui

[2024-10-06 03:24:18 -0700] [26462] [INFO] Starting gunicorn 22.0.0
[2024-10-06 03:24:18 -0700] [26462] [INFO] Listening at: http://127.0.0.1:5000 (26462)
[2024-10-06 03:24:18 -0700] [26462] [INFO] Using worker: sync
[2024-10-06 03:24:18 -0700] [26463] [INFO] Booting worker with pid: 26463
[2024-10-06 03:24:18 -0700] [26464] [INFO] Booting worker with pid: 26464
[2024-10-06 03:24:18 -0700] [26465] [INFO] Booting worker with pid: 26465
[2024-10-06 03:24:18 -0700] [26466] [INFO] Booting worker with pid: 26466
